In [1]:
import pandas as pd 
import numpy as np
import random
import math
result = pd.read_csv('../data/1_before2016.csv').dropna()

X_seq = []
y_seq = []
for name, content in result.groupby('tic'):
    for i in range(len(content.index)-10):
        train_seq = []
        train_label = []
        for j in range(i, i + 10):
            train_seq.append(result.iloc[j]['actual'])
        train_label.append(result.iloc[i+10]['actual'])
        X_seq.append(train_seq)
        y_seq.append(train_label)

result1 = pd.read_csv('../data/1_after2016.csv').dropna()
X_seq1 = []
y_seq1 = []
for name, content in result.groupby('tic'):
    for i in range(len(content.index)-10):
        train_seq = []
        train_label = []
        for j in range(i, i + 10):
            train_seq.append(result.iloc[j]['actual'])
        train_label.append(result.iloc[i+10]['actual'])
        X_seq1.append(train_seq)
        y_seq1.append(train_label)

X = pd.DataFrame(X_seq)
y = pd.DataFrame(y_seq)
Xt = pd.DataFrame(X_seq1)
yt = pd.DataFrame(y_seq1)

In [2]:
N_train = X.shape[0]
N_test = Xt.shape[0]
X_train = X
y_train = y
X_test = Xt
y_test = yt

In [3]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

scaler = StandardScaler()
scaler.fit(X_train)

reg = linear_model.LinearRegression()
reg.fit(scaler.transform(X_train), y_train)

y_lr = reg.predict(scaler.transform(X_test))
ymean = [np.mean(y_test) for i in range(len(y_test))]
mse_lr = mean_squared_error(y_lr, y_test)
r2_lr = r2_score(y_test, y_lr)
r2 = r2_score(y_test, ymean)
print(mse_lr, r2_lr, r2)

0.04694892747111307 0.9078052764444309 0.0


In [5]:
def withinten(y_true, y_pred):
    sum1 = 0
    for i in range(len(y_pred)):
        if(y_pred[i][0] < 1.1*y_true.iloc[i,0] and y_pred[i][0] > 0.9*y_true.iloc[i,0]):
            sum1+=1
    return sum1/len(y_pred)
withinten(y_test, y_lr), withinten(y_test, ymean)

(0.24137363541609524, 0.1217839705963026)

In [6]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
X_train_op = poly.fit_transform(X_train)

from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

X_ridge = X_train_op
N = X_ridge.shape[0]
train = int(0.8*N)
valid = N - train
X_ridge_train = X_ridge[:train]
y_ridge_train = y_train[:train]
X_ridge_valid = X_ridge[train:]
y_ridge_valid = y_train[train:]

scaler1 = StandardScaler()
scaler1.fit(X_ridge_train)
X_ridge_train = scaler1.transform(X_ridge_train)
X_ridge_train[:,0] = np.ones(X_ridge_train.shape[0])
X_ridge_valid = scaler1.transform(X_ridge_valid)
X_ridge_valid[:,0] = np.ones(X_ridge_valid.shape[0])

for lbd in [0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]:
    clf = Ridge(alpha=lbd, solver='lsqr')
    clf.fit(X_ridge_train, y_ridge_train)
    print("lambda=", lbd, ";score=", clf.score(X_ridge_valid, y_ridge_valid))

lambda= 0.01 ;score= 0.9846156553896211
lambda= 0.1 ;score= 0.9847177819171201
lambda= 1 ;score= 0.9831804287459208
lambda= 10 ;score= 0.9757425982794681
lambda= 100 ;score= 0.9559502422750588
lambda= 1000 ;score= 0.9306849005305313
lambda= 10000 ;score= 0.8966761185569536
lambda= 100000 ;score= 0.6981113425159788


In [7]:
from sklearn.pipeline import make_pipeline

clf = make_pipeline(PolynomialFeatures(2), StandardScaler(), Ridge(alpha=0.1, solver='lsqr'))
clf.fit(X_train, y_train)
y_pred_clf = clf.predict(X_test)
clf_loss = mean_squared_error(y_test, y_pred_clf)
r2_clf = r2_score(y_test, y_pred_clf)
print(clf_loss, r2_clf, withinten(y_test, y_pred_clf))

0.009112766155887811 0.98210504474068 0.5280048274727083
